In [17]:
import requests
import pandas as pd
import zipfile
from zipfile import BadZipFile
import io
import os
from tqdm.auto import tqdm
import time
from pathlib import Path

In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/GLAM-Workbench/trove-books/master/trove_digitised_books.csv')

In [25]:
trove_ids = df.loc[df['pages'] == 2][['trove_id', 'pages']].to_dict('records')

In [28]:
output_dir = Path('images', 'ephemera')
output_dir.mkdir(parents=True, exist_ok=True)
for trove_id in tqdm(trove_ids):
    # Check to see if the first page of this issue has already been downloaded
    file_path = Path(output_dir, f'{trove_id["trove_id"]}-1.jpg')
    if not file_path.exists():
        url = f'https://nla.gov.au/{trove_id["trove_id"]}/download?downloadOption=zip&firstPage=0&lastPage={trove_id["pages"] - 1}'
        # Get the file
        r = requests.get(url)
        # The image is in a zip, so we need to extract the contents into the output directory
        try:
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(output_dir)
        except BadZipFile:
            pass
        time.sleep(1)